In [1]:
import tensorflow as tf
import pandas as pd
from IPython.display import clear_output
tf.version

<module 'tensorflow._api.v2.version' from 'C:\\Users\\tpham\\AppData\\Roaming\\Python\\Python38\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>

In [2]:
string = tf.Variable('this is a string', tf.string)
number = tf.Variable(324, tf.int16)
floating = tf.Variable(3.567, tf.float64)

# Tensor Ranks
Rank 0 - is 1 value  
Rank 1 - an array or list  
Rank 2 - a list inside of a list  
to determine the rank of a tensor, it is the deepest level of the nested list

In [6]:
rank2_tensor = tf.Variable([['test', 'rank'], ['two', 'tensor']])

In [7]:
tf.rank(rank2_tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [9]:
#we have 2 lists, with 2 values in each
rank2_tensor.shape

TensorShape([2, 2])

In [12]:
tensor1 = tf.ones([1,2,3]) #creates a tensor of ones in shape 1 list with 2 list and 3 values in each

In [11]:
tensor1

<tf.Tensor: shape=(1, 2, 3), dtype=float32, numpy=
array([[[1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)>

In [13]:
tensor2 = tf.reshape(tensor1, [2,3,1]) #2 lists with 3 lists and 1 value in each

In [14]:
tensor2

<tf.Tensor: shape=(2, 3, 1), dtype=float32, numpy=
array([[[1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.]]], dtype=float32)>

In [15]:
tensor3 = tf.reshape(tensor2, [3,-1]) #3 lists, and inferred next dimension

In [16]:
tensor3

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)>

# Tensor Types
Varirable  
Constant  
Placeholder  
SparseTensor

# Evaluating a Tensor
need to create a session

In [ ]:
with tf.Session() as sess:
    tensor.eval()

In [21]:
t = tf.zeros([5,5,5,5])
t = tf.reshape(t, [125,-1])
#t

In [12]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

In [13]:
dftrain

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
...,...,...,...,...,...,...,...,...,...
622,male,28.0,0,0,10.5000,Second,unknown,Southampton,y
623,male,25.0,0,0,7.0500,Third,unknown,Southampton,y
624,female,19.0,0,0,30.0000,First,B,Southampton,y
625,female,28.0,1,2,23.4500,Third,unknown,Southampton,n


In [20]:
CAT_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']

NUM_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CAT_COLUMNS:
    vocab = dftrain[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocab))
    
for feature_name in NUM_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

In [21]:
feature_columns

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.strin

In [24]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function
        
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

In [27]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)
clear_output()
result['accuracy']

0.7462121

In [28]:
result    

{'accuracy': 0.7462121,
 'accuracy_baseline': 0.625,
 'auc': 0.8241505,
 'auc_precision_recall': 0.7899182,
 'average_loss': 0.533229,
 'label/mean': 0.375,
 'loss': 0.5302901,
 'precision': 0.6333333,
 'prediction/mean': 0.4968389,
 'recall': 0.7676768,
 'global_step': 200}

In [32]:
result = list(linear_est.predict(eval_input_fn))
result[0]['probabilities'][1] # Looking at just 1 prediction, for the prob of death

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Thompson\AppData\Local\Temp\tmpv2ckfndm\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


0.15539272

In [9]:
x = tf.constant([0,1,1,3,5,43,6])
indices = tf.constant([5,3]) # The indices to take from x
tf.gather(x, indices)
tf.gather(x, [5,3])

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([43,  3])>

In [12]:
x = tf.constant([[1,2],
                 [3,4],
                 [5,6]])

x[0,:]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2])>